In [0]:
%sql DESCRIBE HISTORY catalog.schema.table

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
477,2026-01-12T10:41:18.195Z,3655068650007718,dados@korasaude.com.br,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], description -> The 'ldw_fato_financeiro_contas_04' table contains financial data related to patient care. It includes details about the patient, the service provided, the employee who provided the service, and the financial information associated with the service. This data can be used to analyze financial performance, track individual employee performance, and identify trends in patient care. It can also help in identifying potential billing errors and managing patient accounts., isManaged -> false, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)","List(468641714315533, FATO_FINANCEIRO_CONTAS, 819452755888912, 356592655415147, 3655068650007718, manual)",List(3517894581958904),5325-225825-7y451a6x,476,WriteSerializable,false,"Map(numFiles -> 48, numOutputRows -> 55873035, numOutputBytes -> 1720524587)",null,Databricks-Runtime/14.3.x-scala2.12
476,2026-01-11T11:54:52.856Z,3655068650007718,dados@korasaude.com.br,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], description -> The 'ldw_fato_financeiro_contas_04' table contains financial data related to patient care. It includes details about the patient, the service provided, the employee who provided the service, and the financial information associated with the service. This data can be used to analyze financial performance, track individual employee performance, and identify trends in patient care. It can also help in identifying potential billing errors and managing patient accounts., isManaged -> false, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)","List(468641714315533, FATO_FINANCEIRO_CONTAS, 308693972774206, 123737580330385, 3655068650007718, retry)",List(3517894581958904),5325-225825-7y451a6x,475,WriteSerializable,false,"Map(numFiles -> 48, numOutputRows -> 55857289, numOutputBytes -> 1720357136)",null,Databricks-Runtime/14.3.x-scala2.12
475,2026-01-11T10:58:12.714Z,3655068650007718,dados@korasaude.com.br,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], description -> The 'ldw_fato_financeiro_contas_04' table contains financial data related to patient care. It includes details about the patient, the service provided, the employee who provided the service, and the financial information associated with the service. This data can be used to analyze financial performance, track individual employee performance, and identify trends in patient care. It can also help in identifying potential billing errors and managing patient accounts., isManaged -> false, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)","List(468641714315533, FATO_FINANCEIRO_CONTAS, 308693972774206, 888007577750948, 3655068650007718, manual)",List(3517894581958904),5325-225825-7y451a6x,472,WriteSerializable,false,"Map(numFiles -> 54, numOutputRows -> 55857289, numOutputBytes -> 1726247529, conflictDetectionTimeMs -> 83)",null,Databricks-Runtime/14.3.x-scala2.12
474,2026-01-11T10:55:57.295Z,3655068650007718,dados@korasaude.com.br,VACUUM END,Map(status -> COMPLETED),"List(216663820144403, VACUUM, 181028480764844, 445393244707680, 3655068650007718, manual)",List(3627057759689595),5325-225825-7y451a6x,473,SnapshotIsolation,true,"Map(numDeletedFiles -> 422, numVacuumedDirectories -> 1)",null,Databricks-Runtime/14.3.x-scala2.12
473,2026-01-11T10:55:53.164Z,3655068650007718,dados@korasaude.com.br,VACUUM START,"Map(retentionCheckEnabled -> false, defaultRetentionMillis -> 604800000, specifiedRetentionMillis -> 604800000)","List(216663820144403, VACUUM, 181028480764844, 445393244707680, 3655068650007718, manual)",List(3627057759689595),5325-225825-7y451a6x,472,SnapshotIsolation,true,"Map(numFilesToDelete -> 422, si

In [0]:
from datetime import timezone

fechamento_antigo = dbutils.widgets.get("fechamento_antigo") # Parametro via Databricks Jobs
fechamento_novo= spark.sql("""
SELECT max(timestamp) AS ultimo_timestamp
FROM (DESCRIBE HISTORY catalog.schema.table)""").collect()[0]['ultimo_timestamp'].replace(tzinfo=timezone.utc).isoformat(timespec='milliseconds')

In [0]:
df = spark.sql(f"""
select *, to_date('{fechamento_novo}','yyyy-MM-dd')  as dt_fechamento_mudanca 
from catalog.schema.table TIMESTAMP AS OF '{fechamento_novo}'
union all 
select *, to_date('{fechamento_antigo}','yyyy-MM-dd')  as dt_fechamento_mudanca
from catalog.schema.table TIMESTAMP AS OF '{fechamento_antigo}'
""")

In [0]:
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("catalog.schema.table")